In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile
from PIL import Image
from matplotlib.collections import PolyCollection  # remove if unused
from scipy.ndimage import gaussian_filter
from scipy.stats import pearsonr
from skimage import color, filters, io, util
from skimage.measure import label
from skimage.io import imread

In [ ]:

# Function to count the number of segmented regions in an image
def count_segments(image_path):
    # Read the image
    image = tifffile.imread(image_path)
    # Label the connected components
    labeled_image = label(image)
    # Count the number of unique labels, excluding the background (label 0)
    num_segments = len(np.unique(labeled_image)) - 1
    return num_segments

# Directories
segmentations_dir = "IMC_Bodenmiller/OMEandSingleCellMasks/SegmentationsStarDist/"
masks_dir = "IMC_Bodenmiller/OMEandSingleCellMasks/OMEnMasks/Basel_Zuri_masks/"

# Collect TIFF files in the Segmentations folder
seg_files = sorted(f for f in os.listdir(segmentations_dir) if f.endswith(".tiff"))

segmentations_counts_imc = []
masks_counts_imc = []

for seg_file in seg_files:
    seg_path = os.path.join(segmentations_dir, seg_file)

    # Split filename into base + extension
    base_name, ext = os.path.splitext(seg_file)  # e.g. "xxx", ".tiff"
    
    print(base_name)

    # Construct the corresponding _masks filename
    # e.g. "xxx" + "_masks" + ".tiff" -> "xxx_masks.tiff"
    mask_file = base_name + "_maks" + ext
    mask_path = os.path.join(masks_dir, mask_file)
    
    # print(basel_mask_file)
    # print(basel_mask_path)

    # Check if the corresponding file exists in the Basel_Zuri_masks folder
    if not os.path.exists(mask_path):
        print(f"Skipping '{seg_file}' because matching file '{mask_file}' not found.")
        continue

    # Count segments for the segmentation
    seg_count = count_segments(seg_path)
    # Count segments for the Basel_Zuri_mask
    count = count_segments(mask_path)

    segmentations_counts_imc.append(seg_count)
    masks_counts_imc.append(count)

# (Optional) Check if we have equal numbers
if len(segmentations_counts_imc) != len(masks_counts_imc):
    raise ValueError("Mismatch in the number of processed images between the two folders.")


In [ ]:
# Function to count the number of segmented regions in a TIFF image
def count_segments(image_path):
    """
    Count the number of segmented regions in a TIFF image.
    """
    # Read the image
    image = tifffile.imread(image_path)
    # Label the connected components
    labeled_image = label(image)
    # Count the number of unique labels, excluding the background (label 0)
    num_segments = len(np.unique(labeled_image)) - 1
    return num_segments

# Function to count the number of cells in a PNG mask
def count_cells_in_png(image_path, threshold=0):
    """
    Count the number of cells in a PNG mask by thresholding and labeling.
    """
    # Read the PNG image
    image = imread(image_path, as_gray=True)  # Load as grayscale
    # Threshold the image
    binary_mask = image > threshold
    # Label the connected components
    labeled_image = label(binary_mask)
    # Count the number of unique labels, excluding the background (label 0)
    num_cells = len(np.unique(labeled_image)) - 1
    return num_cells

# Directories
segmentations_dir = "MultiplexFluorescentAnnotations/Vectra/SegmentationsStarDist/"
masks_dir = "MultiplexFluorescentAnnotations/Vectra/"

# Collect TIFF files in the Segmentations folder
seg_files = sorted(f for f in os.listdir(segmentations_dir) if f.endswith(".tif"))

segmentations_counts_vectra = []
masks_counts_vectra = []

for seg_file in seg_files:
    seg_path = os.path.join(segmentations_dir, seg_file)

    # Split filename into base + extension
    base_name, ext = os.path.splitext(seg_file)  # e.g., "xxx", ".tiff"
    
    print(f"Processing segmentation: {base_name}")

    # Construct the corresponding ground truth mask filename
    # Replace "-Crop_Tif" with "-Crop_Dapi_Mask_Png" and add ".png"
    ground_truth_file = base_name.replace("-Crop_Tif", "-Crop_Dapi_Mask_Png") + ".png"

    # Search for the ground truth mask file in all subdirectories of masks_dir
    ground_truth_path = None
    for root, _, files in os.walk(masks_dir):
        if ground_truth_file in files:
            ground_truth_path = os.path.join(root, ground_truth_file)
            break
            
            
    if not ground_truth_path:
        ground_truth_file = base_name.replace("-Crop_Tif", "-Crop_Cell_Mask_Png") + ".png"

        # Search for the ground truth mask file in all subdirectories of masks_dir
        ground_truth_path = None
        for root, _, files in os.walk(masks_dir):
            if ground_truth_file in files:
                ground_truth_path = os.path.join(root, ground_truth_file)
                break
                
    
    # If no corresponding ground truth mask is found, skip this file
    if not ground_truth_path:
        print(f"Skipping '{seg_file}' because matching ground truth file '{ground_truth_file}' not found.")
        continue
    else:
        print(f"Found '{ground_truth_file}'")
        

    # Count segments for the segmentation
    seg_count = count_segments(seg_path)
    # Count cells in the ground truth mask
    count = count_cells_in_png(ground_truth_path)
    
    print(f"seg_count '{seg_count}' count '{count}'")
    
    if "P03-10012(56707.13038)650,600" in ground_truth_file:
        print("segmentation mismatch")
        continue
        
    if "P13-10002(45819.10401)2250,500" in ground_truth_file:
        print("unusable ground truth mask")
        continue
        
    segmentations_counts_vectra.append(seg_count)
    masks_counts_vectra.append(count)

# Check if we have equal numbers
if len(segmentations_counts_vectra) != len(masks_counts_vectra):
    raise ValueError("Mismatch in the number of processed images between the two folders.")




In [ ]:

def load_image(path):
    with Image.open(path) as im:
        im_rgb = im.convert("RGB")
        arr = np.array(im_rgb, dtype=np.uint8)
    return arr

def luminance_image(img_rgb):
    # skimage.color.rgb2gray returns float64 in [0,1] from uint8 RGB
    return color.rgb2gray(img_rgb)  # shape (H,W), float64

def edge_strength_gray(gray_img):
    # Sobel magnitude; float64 ~ [0,1]-ish
    sob = filters.sobel(gray_img)
    return float(np.mean(sob))

def noise_level_gray(gray_img, sigma=1.0):
    # Estimate high-frequency residual std
    gray_f = util.img_as_float32(gray_img)  # [0,1]
    blur = gaussian_filter(gray_f, sigma=sigma)
    residual = gray_f - blur
    return float(np.std(residual))

def list_tiffs(folder_path):
    return sorted([
        f for f in os.listdir(folder_path)
        if f.lower().endswith((".tif", ".tiff"))
    ])


def compute_efficiency_dataframe(folders):
    unfiltered_dir = folders["unfiltered"]
    median_dir = folders["median"]
    deep_dir = folders["deep_learning"]

    rows = []

    # iterate over unfiltered filenames as anchors
    uf_files = list_tiffs(unfiltered_dir)

    for fname in uf_files:
        print(fname)
        # unfiltered
        uf_path = os.path.join(unfiltered_dir, fname)

        img_u = load_image(uf_path)
        gray_u = luminance_image(img_u)

        edge_u = edge_strength_gray(gray_u)
        noise_u = noise_level_gray(gray_u)


        # median
        med_path = os.path.join(median_dir, fname)

        img_m = load_image(med_path)
        gray_m = luminance_image(img_m)

        edge_m = edge_strength_gray(gray_m)
        noise_m = noise_level_gray(gray_m)

        # normalized-to-baseline efficiency
        eff_norm_m = (edge_m / edge_u) / (noise_m / noise_u)

        rows.append({
            "file": fname,
            "method": "Median",
            "edge_strength_gray": edge_m,
            "noise_level_gray": noise_m,
            "efficiency_norm": eff_norm_m,
        })


        # deep learning
        dl_path = os.path.join(deep_dir, fname)

        img_d = load_image(dl_path)
        gray_d = luminance_image(img_d)

        edge_d = edge_strength_gray(gray_d)
        noise_d = noise_level_gray(gray_d)

        eff_norm_d = (edge_d / edge_u) / (noise_d / noise_u)

        rows.append({
            "file": fname,
            "method": "Deep learning",
            "edge_strength_gray": edge_d,
            "noise_level_gray": noise_d,
            "efficiency_norm": eff_norm_d,
        })

    return pd.DataFrame(rows)

    
folders = {
    "unfiltered":    "IMC_Bodenmiller/OMEandSingleCellMasks/H&E_unfiltered",
    "median":        "IMC_Bodenmiller/OMEandSingleCellMasks/H&E_median",
    "deep_learning": "IMC_Bodenmiller/OMEandSingleCellMasks/H&E_deep_learning",
}

df_eff = compute_efficiency_dataframe(folders)




In [ ]:

fig, axs = plt.subplots(1, 3, figsize=(12, 4))  # 1 row, 3 columns

# =======================
# Subplot 1: Violin Plot
# =======================

def get_violin_line_x_limits_from_collection(collection, y_val):
    intersections = []
    for path in collection.get_paths():
        vertices = path.vertices
        n = len(vertices)
        for i in range(n - 1):
            v1 = vertices[i]
            v2 = vertices[i + 1]
            y1, y2 = v1[1], v2[1]
            if (y1 - y_val) * (y2 - y_val) < 0:
                x1, x2 = v1[0], v2[0]
                t = (y_val - y1) / (y2 - y1)
                x_intersect = x1 + t * (x2 - x1)
                intersections.append(x_intersect)
            elif y1 == y_val:
                intersections.append(v1[0])
    if intersections:
        return min(intersections), max(intersections)
    else:
        return None, None

ax_efficiency = axs[0]
sns.set_theme(style="ticks")

violin_palette = ["#f2e5ff", "#ffe5f4"]


methods = ["Median", "Deep learning"]

data_list = [
    df_eff.query("method == 'Median'")["efficiency_norm"].dropna().values,
    df_eff.query("method == 'Deep learning'")["efficiency_norm"].dropna().values
]


positions = np.arange(len(data_list))

violin_parts = ax_efficiency.violinplot(
    data_list,
    positions=positions,
    widths=0.9,
    showmeans=False,
    showmedians=False,
    showextrema=False
)

# Style violins
for i, pc in enumerate(violin_parts["bodies"]):
    pc.set_facecolor(violin_palette[i % len(violin_palette)])
    pc.set_edgecolor("black")
    pc.set_alpha(1)

# Axis settings
ax_efficiency.set_xticks(positions)
ax_efficiency.set_xticklabels(methods, fontsize=13)
ax_efficiency.set_title("Enhancement efficiency", fontsize=18)
ax_efficiency.grid(False)
ax_efficiency.minorticks_off()

ax_efficiency.tick_params(axis="y", which="major",
                       direction="out", length=6, width=1.5, color="black")
ax_efficiency.tick_params(axis="y", which="minor",
                       direction="out", length=3, width=1.0, color="black")
ax_efficiency.tick_params(axis="x", which="both",
                       bottom=False, top=False)

# Mean ± SD bars & annotation
for i, m in enumerate(methods):
    data = df_eff[df_eff["method"] == m]["efficiency_norm"].dropna()
    if len(data) == 0:
        continue
    mean_val = data.mean()
    sd_val = data.std()
    pc = violin_parts["bodies"][i]

    x_min_mean, x_max_mean = get_violin_line_x_limits_from_collection(pc, mean_val)
    x_min_hi,   x_max_hi   = get_violin_line_x_limits_from_collection(pc, mean_val + sd_val)
    x_min_lo,   x_max_lo   = get_violin_line_x_limits_from_collection(pc, mean_val - sd_val)

    # fallbacks if that y-level isn't sampled on the outline
    if None in (x_min_mean, x_max_mean):
        x_center = positions[i]
        x_min_mean, x_max_mean = x_center - 0.4, x_center + 0.4
    if None in (x_min_hi, x_max_hi):
        x_min_hi, x_max_hi = x_min_mean, x_max_mean
    if None in (x_min_lo, x_max_lo):
        x_min_lo, x_max_lo = x_min_mean, x_max_mean

    # mean line (black solid)
    ax_efficiency.plot([x_min_mean, x_max_mean], [mean_val, mean_val],
                    color="black", linestyle="-", linewidth=1)
    # +SD and -SD (gray dashed)
    ax_efficiency.plot([x_min_hi, x_max_hi], [mean_val + sd_val, mean_val + sd_val],
                    color="grey", linestyle="--", linewidth=1)
    ax_efficiency.plot([x_min_lo, x_max_lo], [mean_val - sd_val, mean_val - sd_val],
                    color="grey", linestyle="--", linewidth=1)

    # annotate mean
    ax_efficiency.text((x_min_mean + x_max_mean) / 2, mean_val,
                    f"{mean_val:.2f}", ha="center", va="bottom",
                    color="black", fontsize=12)


# =======================
# Subplot 2: Scatter Plot for IMC (Ground Truth on x-axis)
# =======================
ax_imc = axs[1]

# Compute Pearson correlation for IMC
correlation_imc, p_value_imc = pearsonr(masks_counts_imc, segmentations_counts_imc)

sns.scatterplot(
    x=masks_counts_imc,           # ground truth on x-axis
    y=segmentations_counts_imc,   # segmentation counts on y-axis
    alpha=0.1,
    edgecolor=None,
    ax=ax_imc
)
ax_imc.set_xlabel('Number of cells in ground truth', fontsize=13)
ax_imc.set_ylabel('Number of cells in segmentations', fontsize=13)
ax_imc.set_title("IMC", fontsize=18)

# Annotate the correlation result
text_x_imc = 350
text_y_imc = 6300
if p_value_imc < 0.001:
    ax_imc.text(text_x_imc, text_y_imc, f'r = {correlation_imc:.2f}, p < 0.001', fontsize=18, color='black')
else:
    ax_imc.text(text_x_imc, text_y_imc, f'r = {correlation_imc:.2f}, p = {p_value_imc:.2e}', fontsize=18, color='black')

# Set limits and force equal aspect ratio
ax_imc.set_xlim(0, 7000)
ax_imc.set_ylim(0, 7000)
ax_imc.set_aspect('equal', adjustable='box')
ax_imc.set_yticks(np.arange(0, 7000, 2000))
ax_imc.set_xticks(np.arange(0, 7000, 2000))

# =======================
# Subplot 3: Scatter Plot for Multiplex Fluorescence (Ground Truth on x-axis)
# =======================
ax_vectra = axs[2]

# Compute Pearson correlation for Multiplex Fluorescence
correlation_vectra, p_value_vectra = pearsonr(masks_counts_vectra, segmentations_counts_vectra)

sns.scatterplot(
    x=masks_counts_vectra,           # ground truth on x-axis
    y=segmentations_counts_vectra,   # segmentation counts on y-axis
    alpha=0.3,
    edgecolor=None,
    ax=ax_vectra
)
ax_vectra.set_xlabel('Number of cells in ground truth', fontsize=13)
ax_vectra.set_ylabel('Number of cells in segmentations', fontsize=13)
ax_vectra.set_title("Multiplex Fluorescence", fontsize=18)

text_x_vectra = 45
text_y_vectra = 810
if p_value_vectra < 0.001:
    ax_vectra.text(text_x_vectra, text_y_vectra, f'r = {correlation_vectra:.2f}, p < 0.001', fontsize=18, color='black')
else:
    ax_vectra.text(text_x_vectra, text_y_vectra, f'r = {correlation_vectra:.2f}, p = {p_value_vectra:.2e}', fontsize=18, color='black')

ax_vectra.set_xlim(0, 900)
ax_vectra.set_ylim(0, 900)
ax_vectra.set_aspect('equal', adjustable='box')
ax_vectra.set_yticks(np.arange(0, 900, 200))
ax_vectra.set_xticks(np.arange(0, 900, 200))


plt.tight_layout()
plt.savefig("figures/combined_plots.pdf", format="pdf", bbox_inches='tight')
plt.show()
